<p style="text-align:center; font-size:300%; line-height:300%; font-weight:bold; background-color:#2F52A3; color:white">Algorithme du pivot de Gauss</p>

<div style="font-weight:bold; background-color:#61C3CE">
    Evaluation :
    <ul>
        <li>Résoudre des systèmes linéaires</li>
    </ul>
</div>

**Recommandations :**

+ Respectez rigoureusement la signature des fonctions (casse incluse), et les consignes.
+ Copiez-collez le code de votre fonction (ne contenant aucun print ou autre effet de bord) sur Moodle (CodeRunner)
+ Utilisez les alias définis ci-dessous.
+ Ne faites pas d'import dans Moodle, c'est déjà fait !
+ Même si vous travaillez à plusieurs, faites un dépôt par personne.
+ Il n'y aura aucune "démo" dans les cinq dernières minutes. Ce travail et les tests Moodle seront pris en compte dans la note finale.
+ Vous allez coder **des fonctions**; cela signifie que vous **retournez** le résultat et que vous ne modifiez pas les paramètres (**pas d'effet de bord**).

**Objectif :**

+ Dans ce TP, vous allez terminer le travail mis en place à la séance précédente en implémentant l'algorithme de Gauss permettant de résoudre des systèmes linéaires.

In [43]:
# Exécutez cette cellule, vous pourrez alors appeler ces fonctions dans le reste de ce notebook
import numpy as np

def Echange(i, j , M):
    copy = np.copy(M)
    copy[i,:] = M[j,:]
    copy[j,:] = M[i,:]
    return copy

def CombinaisonLineaire(i, l, j , A):
    M = np.copy(A)
    M[i,:] = A[i,:] + l*A[j,:]
    return M

def ChoixPivot(j,A):
    nbLignes = A.shape[0]
    i = j;
    while (i < nbLignes and A[i,j]==0):
        i = i + 1
    if (i == nbLignes):
        i=-1
    return i

<p style="font-size:150%; line-height:150%; font-weight:bold; background-color:#FFDD00">Exercice : Fonction d'échelonnement</p>

Si $A$ est la **matrice augmentée** d'un système linéaire, écrivez une fonction `Echelonne(A)` qui retourne la matrice échelonnée du système.

**Par exemple, pour le système :**

$$\left\{\begin{matrix}
2x & + & 3y & - & z  & = & -7 \\
5x & - & 2y & + & 3z & = & 18 \\
-x & + & 7y & + & 8z & = &  9
\end{matrix}\right.$$

La matrice augmentée $A$ est :
$$\left(\begin{matrix}
2  &  3 & -1 & -7 \\
5  & -2 &  3 & 18 \\
-1 &  7 &  8 &  9
\end{matrix}\right)$$

Et `Echelonne(A)` retourne :

$$\left(\begin{matrix}
2 &  3   & -1    & -7 \\
0 & -9.5 &  5.5  & 35.5 \\
0 &  0   & 12.42 & 37.26
\end{matrix}\right)$$

De plus, la solution de ce système est $(1;-2;3)$

**Deuxième exemple, le système :**

$$\left\{\begin{matrix}
x & + & y & + & z  & = & -1 \\
-x & + & y & + & z & = & 1 \\
2x & - & y & + & z & = &  -6
\end{matrix}\right.$$

La matrice augmentée $A$ est :
$$\left(\begin{matrix}
 1  &  1 &  1 & -1 \\
-1  &  1 &  1 &  1 \\
 2  & -1 &  1 & -6
\end{matrix}\right)$$

Et `Echelonne(A)` retourne :

$$\left(\begin{matrix}
1 &  1 &  1 & -1 \\
0 &  2 &  2 &  0 \\
0 &  0 &  2 & -4
\end{matrix}\right)$$

De plus, la solution de ce système est $(-1;2;-2)$

**Indications :**
+ Il est bien sûr vivement recommandé d'utiliser les fonctions précédentes !
+ Tester ses fonctions sur seulement deux exemples, n'est clairement pas suffisant !


In [44]:
def Echelonne(A):
    """
    param A:np.array
        /matrice correspondant à un système linéaire
    return :np.array
        /système linéaire échelonné
        
        |x, y, Z=?|
        | , y, Z=?|
        | ,  , Z=?|
    """
    copy = np.copy(A)
    for i in range(len(A)-1):
        lignepivot = ChoixPivot(i, copy)
        Echange(i-1, lignepivot, copy)
        pivot = copy[i+1, i]
        for j in range(i+1, len(A)):
            coeff = copy[j, i]/ copy[i, i]
            copy = CombinaisonLineaire(j, -coeff, i , copy)
    return np.around(copy, 8)

<p style="font-size:150%; line-height:150%; font-weight:bold; background-color:#FFDD00">Exercice : Résolution</p>

Si $A$ est la matrice augmentée d'un système linéaire, écrivez une fonction `Gauss(A)` qui retourne la solution du système. 

**Indications :**
+ Ce système est supposé de Cramer. Aucun test n'est effectué pour le vérifier.
+ Etant donnée une équation $a_{i,i}x_{i} + a_{i,i+1}x_{i+1} + \cdots + a_{i,n}x_{n} = b_i$.
Si l'on connaît $x_{i+1},\ldots,x_n$, alors on trouve $x_i$ avec la formule :

$$x_{i} = \frac{1}{a_{i,i}}\left(b_i - \sum_{k=i+1}^n a_{i,k}x_k\right)$$

+ L'essentiel de votre travail consiste à coder la formule précédente.
+ Mais, commencez par échelonner !
+ Testez votre fonction avec les deux exemples de la partie précédente, mais ça ne suffit pas !

In [49]:
def Gauss(A):
    """
    param A:np.array
        /matrice correspondant à un système linéaire
    return :np.array
        /solution du système linéaire
    """
    copy = np.copy(Echelonne(A))
    
    #reduction du système linéaire
    for i in range(len(copy)-1, -1, -1):
        for j in range(i-1, -1, -1):
            if copy[j, i] != 0:
                coeff = -(copy[j, i]/(copy[i, i]))
                copy = CombinaisonLineaire(j, coeff, i, copy)
    """
    après cette partie de code:
        |x,  ,  =?|
        | , y,  =?|
        | ,  , Z=?|
    """
    #solution unique
    return np.array([copy[k, -1]/copy[k, k] for k in range(len(A))])
    #je ne sais pas trop si il fallait prendre en compte les cas de solution
    #tel que les solution infini ou les solution nulle. Du coup je ne les ai
    #pas traité

[ 1. -2.  3.]
[-1.  2. -2.]


<p style="font-size:150%; line-height:150%; font-weight:bold; color:#D67D3A">Attention : </p>

Les fonctions à rendre pour ce TP sont :
+ `Echelonne(A)`
+ `Gauss(A)`